# Time Series Prediction


## Introduction

Predicting time series data is useful for a variety of business applications. Companies forecast time series data in order to forecast sales, manage their supply chains, and more.

LSTMs are commonly used for time series prediction because they are able to learn the underlying patterns of time series data. 

In this exercise, we will use an LSTM to forecast the data from a random timeseries by training it on the timeseries.

## Instructions

You can begin forecasting trends with LSTMs with the following steps:

1. Generate your timeseries data in Numpy
2. Load the data into PyTorch
3. Train your model on the data
4. Run the model to predict the next steps in the time series.

## Example Code

<b> Generate X, Y data </b>

In [2]:
# Create a sinusoidal dataset

import numpy as np
X = np.arange(1000)
Y = np.array([np.sin(x/10) + 0.01*x for x in X])

In [23]:
seq_length = 20

In [24]:
Y

array([  0.00000000e+00,   1.09833417e-01,   2.18669331e-01,
         3.25520207e-01,   4.29418342e-01,   5.29425539e-01,
         6.24642473e-01,   7.14217687e-01,   7.97356091e-01,
         8.73326910e-01,   9.41470985e-01,   1.00120736e+00,
         1.05203909e+00,   1.09355819e+00,   1.12544973e+00,
         1.14749499e+00,   1.15957360e+00,   1.16166481e+00,
         1.15384763e+00,   1.13630009e+00,   1.10929743e+00,
         1.07320937e+00,   1.02849640e+00,   9.75705212e-01,
         9.15463181e-01,   8.48472144e-01,   7.75501372e-01,
         6.97379880e-01,   6.14988150e-01,   5.29249329e-01,
         4.41120008e-01,   3.51580662e-01,   2.61625857e-01,
         1.72254306e-01,   8.44588980e-02,  -7.83227690e-04,
        -8.25204433e-02,  -1.59836141e-01,  -2.31857891e-01,
        -2.97766159e-01,  -3.56802495e-01,  -4.08277111e-01,
        -4.51575772e-01,  -4.86165937e-01,  -5.11602074e-01,
        -5.27530118e-01,  -5.33691004e-01,  -5.29923258e-01,
        -5.16164609e-01,

<b> Create an LSTM Model </b>

In [41]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    
    def __init__(self,input_size, output_size,hidden_size ,num_layers):
        
        super(LSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.lstmcell = nn.LSTMCell(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self,X,hidden):
        batch_size = X.size(0)
        
        # get RNN outputs
        r_out, hidden = self.lstmcell(X, hidden)
        # shape output to be (batch_size*seq_length, hidden_dim)
        r_out = r_out.view(-1, self.hidden_dim)  
        
        # get final output 
        pred = self.fc(r_out)
        
       
        return pred

In [42]:
# decide on hyperparameters
input_size=1 
output_size=1
hidden_dim=32
num_layers =1

model = LSTM(input_size,output_size,hidden_dim ,num_layers)

In [43]:
model

LSTM(
  (lstmcell): LSTMCell(1, 32)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)

In [44]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [45]:
def train(model, n_steps, print_every):
    
    # initialize the hidden state
    hidden = None      
    
    for batch_i, step in enumerate(range(n_steps)):
        # defining the training data 
        time_steps = np.linspace(step * np.pi, (step+1)*np.pi, seq_length + 1)
        data = np.sin(time_steps)
        data.resize((seq_length + 1, 1)) # input_size=1

        X = data[:-1]
        Y = data[1:]
        
        # convert data into Tensors
        x_tensor = torch.Tensor(X).unsqueeze(0) # unsqueeze gives a 1, batch_size dimension
        y_tensor = torch.Tensor(Y)

        # outputs from the rnn
        prediction, hidden = model(x_tensor, hidden)

        ## Representing Memory ##
        # make a new variable for hidden and detach the hidden state from its history
        # this way, we don't backpropagate through the entire history
        hidden = hidden.data

        # calculate the loss
        loss = criterion(prediction, y_tensor)
        # zero gradients
        optimizer.zero_grad()
        # perform backprop and update weights
        loss.backward()
        optimizer.step()

        # display loss and predictions
        if batch_i%print_every == 0:        
            print('Loss: ', loss.item())
            plt.plot(time_steps[1:], X, 'r.') # input
            plt.plot(time_steps[1:], prediction.data.numpy().flatten(), 'b.') # predictions
            plt.show()
    
    return model

In [46]:
n_steps = 75
print_every = 15

trained_model = train(model, n_steps, print_every)

RuntimeError: input has inconsistent input_size: got 20, expected 1